In [15]:
import os
import numpy as np
import asyncio
import re
import fitz # PyMuPDF
from concurrent.futures import ThreadPoolExecutor
import torch

In [16]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from pymilvus import connections, utility

In [17]:
from langchain.agents import Tool, initialize_agent
# from langchain_community.chat_models import ChatOllama
from langchain_ollama import OllamaLLM, ChatOllama
from llama_index.llms.ollama import Ollama
from pymilvus import CollectionSchema, FieldSchema, DataType, Collection

In [18]:
# from llama_index.readers.file import PDFReader  # 重要修正点
# from pdfminer import text_encoding
from llama_index.readers.file import PyMuPDFReader
from time import time
import spacy
from spacy.lang.zh import Chinese

from llama_index.core.schema import Document

In [19]:
from llama_index.core.query_engine import RetrieverQueryEngine

In [6]:
def iniDeepSeek():
    model: str = "deepseek-r1:7b",
    base_url: str = "http://localhost:11434"
    return OllamaLLM(
                model=model,
                base_url=base_url,
                streaming=False)

#### version 3
- Customized Chunk

In [7]:
# 配置 Milvus 本地連接
connections.connect("default", host="localhost", port="19530")

In [8]:
# 全局設定本地 LLM 模型 (重要修正點)
Settings.llm = Ollama(model="deepseek-r1:7b", temperature=0.2)
# Settings.embed_model = HuggingFaceEmbedding(
#     model_name="shibing624/text2vec-base-chinese",
#     cache_folder="../../../../../Embedding_Models/",
#     embed_batch_size=32,
#     device="cpu"
# )
# 全局設定
Settings.embed_model = HuggingFaceEmbedding(
    model_name="shibing624/text2vec-base-chinese",
    cache_folder="../../../../../Embedding_Models/",
    embed_batch_size=64,  # 加大批量處理
    device="cuda" if torch.cuda.is_available() else "cpu"
)
# 初始化 spaCy 中文模型
nlp = spacy.load("zh_core_web_sm", disable=["ner", "parser"])
nlp.add_pipe("sentencizer")

#### Customized Chunk Classes

In [13]:
class PDFProcessorOptimized:
    """高效PDF處理器"""
    def __init__(self, chunk_size=768, timeout=300):
        self.chunk_size = chunk_size
        self.timeout = timeout
        self.section_pattern = re.compile(
            r'第[一二三四五六七八九十百]+條\s+', 
            re.UNICODE
        )

    def _extract_text(self, file_path):
        """帶超時機制的文本提取"""
        start_time = time.time()
        doc = fitz.open(file_path)
        text = ""
        
        for page in doc:
            if time.time() - start_time > self.timeout:
                raise TimeoutError(f"處理文件 {file_path} 超時")
            text += page.get_text("text") + "\n"
        
        return self._clean_text(text)

    def _clean_text(self, text):
        """高效文字清理"""
        text = re.sub(r'[\u3000\xa0\s]+', ' ', text)
        return re.sub(r'\n{3,}', '\n', text).strip()

    def _chunk_with_timeout(self, text):
        """分塊處理超時保護"""
        from multiprocessing import get_context
        
        def _chunk_task(t):
            sections = self.section_pattern.split(t)
            return [s.strip() for s in sections if s.strip()]
        
        with get_context("spawn").Pool(1) as pool:
            future = pool.apply_async(_chunk_task, (text,))
            try:
                return future.get(timeout=self.timeout//2)
            except Exception:
                raise TimeoutError("分塊處理超時")

#### parallel pdf processing

In [ ]:
class ParallelPDFHandler:
    """並行PDF處理控制器"""
    def __init__(self, max_workers=None, chunk_size=768):
        self.executor = ProcessPoolExecutor(
            max_workers=max_workers or os.cpu_count()//2
        )
        self.processor = PDFProcessorOptimized(chunk_size=chunk_size)

    def batch_process(self, pdf_dir, timeout=600):
        """批次處理PDF目錄"""
        tasks = {}
        pdf_files = [
            os.path.join(pdf_dir, f) 
            for f in os.listdir(pdf_dir) 
            if f.lower().endswith(".pdf")
        ]
        
        # 提交任務
        for file_path in pdf_files:
            future = self.executor.submit(
                self.processor.process_file, 
                file_path
            )
            tasks[future] = file_path
        
        # 收集結果
        documents = []
        for future in as_completed(tasks, timeout=timeout):
            try:
                documents.extend(future.result())
            except Exception as e:
                print(f"處理失敗 {tasks[future]}: {str(e)}")
        
        return documents

#### Milvus Optimizer

In [ ]:
# Milvus配置強化版
class MilvusOptimizer:
    def __init__(self, host="localhost", port="19530"):
        connections.connect("default", host=host, port=port)
        
        self.embed_model = HuggingFaceEmbedding(
            model_name="shibing624/text2vec-base-chinese",
            embed_batch_size=256,
            device="cuda" if torch.cuda.is_available() else "cpu"
        )
    
    def get_index(self, collection_name, dim=768):
        if utility.has_collection(collection_name):
            return MilvusVectorStore(
                collection_name=collection_name,
                dim=dim,
                overwrite=False
            )
        
        return MilvusVectorStore(
            collection_name=collection_name,
            dim=dim,
            overwrite=True,
            index_params={
                "metric_type": "L2",
                "index_type": "IVF_PQ",
                "params": {"nlist": 2048, "m": 32}
            },
            search_params={"nprobe": 32}
        )

#### main function

In [ ]:
def build_rag_system(pdf_dir, collection_name="taiwan_law_pro"):
    """強化版RAG建置流程"""
    # 1. 並行處理PDF
    start_time = time.time()
    handler = ParallelPDFHandler(chunk_size=768)
    documents = handler.batch_process(pdf_dir)
    print(f"📚 文本處理完成 耗時: {time.time()-start_time:.2f}s")
    
    # 2. 向量儲存配置
    optimizer = MilvusOptimizer()
    vector_store = optimizer.get_index(collection_name)
    storage = StorageContext.from_defaults(vector_store=vector_store)
    
    # 3. 分批建立索引
    batch_size = 500
    for i in range(0, len(documents), batch_size):
        VectorStoreIndex.from_documents(
            documents=documents[i:i+batch_size],
            storage_context=storage,
            embed_model=optimizer.embed_model,
            show_progress=True
        )
    
    # 4. 效能優化
    utility.compact(collection_name)
    vector_store.load()
    
    return VectorStoreIndex.from_vector_store(vector_store).as_query_engine(
        similarity_top_k=3,
        response_mode="tree_summarize"
    )

#### original codes

In [9]:
# ---------- 1. 法律文件分块处理器 ----------
class LegalChunkOptimizer:
    """法律文件分块优化器"""
    def __init__(self, chunk_size=768, overlap=128):
        self.chunk_size = chunk_size
        self.overlap = overlap
        self.section_pattern = re.compile(
            r'第[一二三四五六七八九十百]+條\s+', 
            re.UNICODE
        )
        self.article_pattern = re.compile(
            r'(前項|本條|前款|右列)\s', 
            re.UNICODE
        )

    def _legal_sentence_split(self, text):
        """法律条文感知的句子分割"""
        doc = nlp(text)
        sentences = []
        for sent in doc.sents:
            # 处理法律条文中的特殊连接词
            if self.article_pattern.search(sent.text):
                sentences[-1] += " " + sent.text
            else:
                sentences.append(sent.text)
        return sentences

    def chunk_document(self, text):
        print("原始文本样本：", text[:500])  # 打印前500字符
        """混合分块策略"""
        chunks = []
        
        # 第一阶段：按法律条文分块
        sections = self.section_pattern.split(text)
        sections = [s.strip() for s in sections if s.strip()]
        print("匹配到的法律条文数：", len(sections))
        
        # 第二阶段：递归分块
        for sec in sections:
            buffer = ""
            sentences = self._legal_sentence_split(sec)
            
            for sent in sentences:
                if len(buffer) + len(sent) > self.chunk_size:
                    chunks.append(buffer.strip())
                    buffer = buffer[-self.overlap:] + " " + sent
                else:
                    buffer += " " + sent
            if buffer:
                chunks.append(buffer.strip())
        
        return chunks

# ---------- 2. PDF 并行处理器 ----------
class PDFParallelProcessor:
    """PDF 并行处理引擎"""
    def __init__(self, chunk_size=768):
        self.chunker = LegalChunkOptimizer(chunk_size=chunk_size)
        
    def _process_single_pdf(self, file_path):
        """处理单个 PDF 文件"""
        doc = fitz.open(file_path)
        full_text = ""
        for page_num, page in enumerate(doc):
            text = page.get_text("text")
            print(f"📄 {file_path} 第{page_num+1}页原始文本：\n{text[:200]}...")  # 打印前200字符
        
        # 提取并清理文本
        for page in doc:
            text = page.get_text("text")
            text = re.sub(r'[\u3000\xa0]+', ' ', text)  # 处理特殊空格
            text = re.sub(r'\n{3,}', '\n\n', text)      # 合并多余换行
            full_text += text + "\n"
            
        chunks = self.chunker.chunk_document(full_text)
        print(f"🔖 {file_path} 分块结果：{len(chunks)} chunks")
        
        return chunks

    def process_batch(self, pdf_dir, workers=4):
        """批量处理 PDF 目录"""
        pdf_files = [
            os.path.join(pdf_dir, f) 
            for f in os.listdir(pdf_dir) 
            if f.lower().endswith(".pdf")
        ]
        print(f"pdf_files:{pdf_files}")
        with ThreadPoolExecutor(max_workers=workers) as executor:
            results = list(executor.map(self._process_single_pdf, pdf_files))
        
        return [Document(text=c) for sublist in results for c in sublist]

# ---------- 3. Milvus 向量存储配置 ----------
class MilvusConfigurator:
    """Milvus 配置管理器"""
    def __init__(self, host="localhost", port="19530"):
        connections.connect("default", host=host, port=port)
        
        # 嵌入模型配置
        Settings.embed_model = HuggingFaceEmbedding(
            model_name="shibing624/text2vec-base-chinese",
            cache_folder="../../../../../Embedding_Models/",
            embed_batch_size=128,
            device="cuda" if torch.cuda.is_available() else "cpu"
        )
    
    def get_vector_store(self, collection_name):
        """智能获取向量存储"""
        if utility.has_collection(collection_name):
            print(f"Loading existing collection: {collection_name}")
            return MilvusVectorStore(
                uri="http://localhost:19530",
                collection_name=collection_name,
                dim=768,
                overwrite=False
            )
        else:
            print(f"Creating new collection: {collection_name}")
            return MilvusVectorStore(
                uri="http://localhost:19530",
                collection_name=collection_name,
                dim=768,
                overwrite=True,
                index_params={
                    "metric_type": "L2",
                    "index_type": "IVF_FLAT",
                    "params": {"nlist": 2048}
                },
                search_params={"nprobe": 64}
            )

# ---------- 4. 主执行流程 ----------
def build_legal_rag_system(pdf_dir, collection_name="taiwan_law_v4"):
    # 初始化配置
    milvus_mgr = MilvusConfigurator()
    processor = PDFParallelProcessor(chunk_size=768)
    
    # 处理 PDF 文件
    print("⏳ 开始处理法律文件...")
    documents = processor.process_batch(pdf_dir)
    print(f"✅ 成功处理 {len(documents)} 个文本分块")
    
    # 配置 Milvus
    vector_store = milvus_mgr.get_vector_store(collection_name)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    # 建立索引
    print("⏳ 建立向量索引...")
    index = VectorStoreIndex.from_documents(
        documents=documents,
        storage_context=storage_context,
        show_progress=True,
        batch_size=256
    )
    
    # 持久化存储
    storage_context.persist(persist_dir=f"./storage/{collection_name}")
    print(f"📦 索引已持久化到 ./storage/{collection_name}")
    
    return index.as_query_engine(
        similarity_top_k=3,
        vector_store_kwargs={"search_params": {"nprobe": 64}},
        llm=Settings.llm
    )



In [10]:
class AdaptiveChunker:
    """基於內容複雜度動態調整分塊"""
    def __init__(self):
        self.complexity_thresholds = {
            'low': (1024, 64),
            'medium': (768, 128),
            'high': (512, 192)
        }
    
    def analyze_complexity(self, text):
        # 計算句子長度變異數
        sentences = sent_tokenize(text)
        lengths = [len(s) for s in sentences]
        var = np.var(lengths)
        return 'high' if var > 1000 else 'medium' if var > 500 else 'low'
    
    def chunk(self, text):
        complexity = self.analyze_complexity(text)
        size, overlap = self.complexity_thresholds[complexity]
        return self._chunk_with_params(text, size, overlap)

In [20]:
# 异步查询处理器
class AsyncQueryEngine:
    def __init__(self, query_engine):
        self.query_engine = query_engine
        
    async def async_query(self, question):
        return await self.query_engine.aquery(question)

# 优化后的查询配置
def optimize_query_engine(index):
    return RetrieverQueryEngine(
        retriever=index.as_retriever(
            similarity_top_k=3,
            vector_store_kwargs={
                "search_params": {
                    "nprobe": 32,  # 降低精度提升速度
                    "timeout": 30   # 增加超时限制
                }
            }
        ),
        response_mode="compact"
    )
# 初始化异步引擎
query_engine = optimize_query_engine(legal_index)
async_engine = AsyncQueryEngine(query_engine)

# 执行异步查询
async def execute_query():
    try:
        response = await async_engine.async_query(
            "請解釋台灣民法中的不可抗力條款，並列出相關法條編號"
        )
        print(response)
    except Exception as e:
        print(f"查询失败: {str(e)}")

# 运行事件循环
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(execute_query())

NameError: name 'legal_index' is not defined

In [ ]:
# 执行异步查询
async def execute_query():
    try:
        response = await async_engine.async_query(
            "請解釋台灣民法中的不可抗力條款，並列出相關法條編號"
        )
        print(response)
    except Exception as e:
        print(f"查询失败: {str(e)}")

In [11]:
def run_custom_chunk():
    # 初始化 RAG 系统
    query_engine = build_legal_rag_system(
        pdf_dir="../../../misc/law/pdf/",
        collection_name="taiwan_law_v4"
    )
    
    # 测试查询
    response = query_engine.query(
        "請解釋台灣民法中的不可抗力條款，並列出相關法條編號"
    )
    print("📜 法律檢索結果:", response)a

In [12]:
run_custom_chunk()

⏳ 开始处理法律文件...
pdf_files:['../../../misc/law/pdf/刑法.pdf', '../../../misc/law/pdf/民事訴訟法.pdf', '../../../misc/law/pdf/民法.pdf', '../../../misc/law/pdf/刑事訴訟法.pdf']
📄 ../../../misc/law/pdf/刑法.pdf 第1页原始文本：
輔大法服刑法法條 
1 
 
刑法 
目錄 
刑法 
3 
第 一 章 法例 
3 
第 二 章 刑事責任 
7 
第 三 章 未遂犯 
9 
第 四 章 正犯與共犯 
9 
第 五 章 刑 
10 
第 五 章之一 沒收 
12 
第 五 章之二 易刑 
14 
第 六 章 累犯 
16 
第 七 章 數罪併罰 
17 
第 八 章 刑之酌科及加減 
18 
第 九 章 緩刑 
21 
第 十 ...
📄 ../../../misc/law/pdf/刑法.pdf 第2页原始文本：
輔大法服刑法法條 
2 
 
第 二十五 章 遺棄罪 
71 
第 二十六 章 妨害自由罪 
72 
第 二十七 章 妨害名譽及信用罪 
75 
第 二十八 章 妨害秘密罪 
76 
第 二十八 章之一 妨害性隱私及不實性影像罪 
78 
第 二十九 章 竊盜罪 
80 
第 三十 章 搶奪強盜及海盜罪 
81 
第 三十一 章 侵占罪 
83 
第 三十二 章 詐欺背信及重利罪 
84 
第 三...
📄 ../../../misc/law/pdf/刑法.pdf 第3页原始文本：
輔大法服刑法法條 
3 
 
刑法 
 
修正日期：民國 112 年 12 月 27 日 
 
第 一 章 法例 
 
第 1 條 
行為之處罰，以行為時之法律有明文規定者為限。拘束人身自由之保安處分，亦同。 
 
第 2 條 
Ⅰ 行為後法律有變更者，適用行為時之法律。但行為後之法律有利於行為人者，適用最有利
於行為人之法律。 
Ⅱ 沒收、非拘束人身自由之保安處分適用裁判時之法律。 
Ⅲ 處罰或...
📄 ../../../misc/law/pdf/刑法.pdf 第4页原始文本：
輔大法服刑法法條 
4 
 
第 5 條 
本法於凡在中華民國領域外犯下列各罪者，適用之： 
一、內亂罪。 
二、外患罪。 
三、第一百三

2025-04-15 17:39:54,302 [DEBUG][_create_connection]: Created new connection using: 4eda9af0487e4a93923362b1b6b8a5a9 (async_milvus_client.py:600)


🔖 ../../../misc/law/pdf/民法.pdf 分块结果：233 chunks
✅ 成功处理 619 个文本分块
Loading existing collection: taiwan_law_v4
⏳ 建立向量索引...


Parsing nodes:   0%|          | 0/619 [00:00<?, ?it/s]

Some nodes are missing content, skipping them...


Generating embeddings:   0%|          | 0/656 [00:00<?, ?it/s]

📦 索引已持久化到 ./storage/taiwan_law_v4


ReadTimeout: timed out

In [ ]:
def process_pdf(file_path, chunker):
    """使用 spaCy 优化的 PDF 处理流程"""
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    
    # 法律文件特定清理
    text = re.sub(r'(\n\s*)+', '\n', text)  # 合并多余换行
    text = re.sub(r'\u3000', ' ', text)     # 处理全角空格
    
    return chunker.legal_chunking(text)



#### load pdf